🔧 **Setup Required**: Before running this notebook, please follow the [setup instructions](../README.md#setup-instructions) to configure your environment and API keys.

## Building custom components with Haystack

Whereas the Haystack library provides a wide range of pre-built components, it is also possible to build custom components. This notebook demonstrates how to build a custom component for Haystack.

The custom component we will build is a simple one: a component that takes a list of strings as input and returns the number of words in each string. This is a simple example, but it demonstrates the basic principles of building a custom component.

In [30]:
from typing import List
from haystack import component, Pipeline

from haystack import component
from typing import List

@component
class Prefixer:
    """
    A component that adds a predefined prefix to each string in a list.
    """
    def __init__(self, prefix: str = ">> "):
        self.prefix = prefix

    @component.output_types(prefixed_documents=List[str])
    def run(self, documents: List[str]):
        """
        Adds the prefix to a list of documents.
        """
        processed_docs = [self.prefix + doc for doc in documents]
        return {"prefixed_documents": processed_docs}


In [31]:
prefixer_instance = Prefixer(prefix=">> ")  
prefixer_instance.run(documents=["Document one.", "Document two."])  

{'prefixed_documents': ['>> Document one.', '>> Document two.']}